In [1]:
import pandas as pd
import mysql.connector
import plotly.express as px
from datetime import date
from pathlib import Path


In [2]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
connection = mysql.connector.connect(
    host='localhost',  # Specify the host of the MySQL server
    user='root',  # Specify your MySQL username
    password='Combination2#',  # Specify your MySQL password
    database='sales_data'  # Specify the name of your MySQL database
)


In [4]:
# Example query to read all data from a table named 'your_table'
query = "SELECT * FROM customers"
df = pd.read_sql(query, connection)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28440\225072115.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [5]:
# Display the first few rows of the DataFrame
print(df.head())


   customer_id first_name last_name                   email     phone
0            1       John       Doe     johndoe@example.com  555-1234
1            2       Jane       Doe     janedoe@example.com  555-5678
2            3        Bob     Smith    bobsmith@example.com  555-9012
3            4      Alice     Jones  alicejones@example.com  555-3456
4            5      David     Brown  davidbrown@example.com  555-7890


>## Total Sales by Month

In [6]:
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales_data.sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df_sales = pd.read_sql_query(query, connection)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28440\1339223681.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales = pd.read_sql_query(query, connection)


In [7]:
df_sales

,sale_date,total_sales
0,2022-01-02,660.0
1,2022-01-03,915.0
2,2022-01-04,1300.0
3,2022-01-05,960.0
4,2022-01-06,1095.0
...,...,...
335,2022-12-26,880.0
336,2022-12-28,1355.0
337,2022-12-29,1150.0
338,2022-12-30,835.0


In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sale_date    340 non-null    object 
 1   total_sales  340 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.4+ KB


In [9]:
# Convert sale_date to datetime
df_sales['sale_date'] = pd.to_datetime(df_sales['sale_date'])
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    340 non-null    datetime64[ns]
 1   total_sales  340 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.4 KB


In [10]:
# Set the sale_date column as the index
df_sales = df_sales.set_index('sale_date')
df_sales.sample(10)

,total_sales
sale_date,
2022-08-15,280.0
2022-10-26,1605.0
2022-06-25,1070.0
2022-10-09,1045.0
2022-01-21,600.0
2022-11-08,990.0
2022-11-11,750.0
2022-10-30,380.0
2022-11-21,585.0


In [11]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df_sales.resample('M').sum()
df_monthly

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28440\1898053559.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df_sales.resample('M').sum()


,total_sales
sale_date,
2022-01-31,18065.0
2022-02-28,20915.0
2022-03-31,25825.0
2022-04-30,22485.0
2022-05-31,25735.0
2022-06-30,24800.0
2022-07-31,33305.0
2022-08-31,19850.0
2022-09-30,19880.0


In [12]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,18065.0,Jan
2022-02-28,20915.0,Feb
2022-03-31,25825.0,Mar
2022-04-30,22485.0,Apr
2022-05-31,25735.0,May
2022-06-30,24800.0,Jun
2022-07-31,33305.0,Jul
2022-08-31,19850.0,Aug
2022-09-30,19880.0,Sep


In [13]:
# Create the Plotly figure with text parameter
plotly_template = "presentation"
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png',
                width=1200,
                height=400,
                scale=4)

> ## Total Sales by Product

In [15]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales_data.sales s
JOIN sales_data.products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df_product = pd.read_sql_query(query, connection)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28440\30097759.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [16]:
df_product

,product_name,total_sales
0,Product A,57200.0
1,Product B,23025.0
2,Product C,83100.0
3,Product D,43880.0
4,Product E,73860.0


In [18]:
fig = px.bar(df_product,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product name',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png',
                width=1200,
                height=400,
                scale=4)

>## Top Customer by Sales

In [24]:
query = """select * from sales_data.customers;"""
customer = df_customer = pd.read_sql_query(query, connection)
customer

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28440\2547467950.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,customer_id,first_name,last_name,email,phone
0,1,John,Doe,johndoe@example.com,555-1234
1,2,Jane,Doe,janedoe@example.com,555-5678
2,3,Bob,Smith,bobsmith@example.com,555-9012
3,4,Alice,Jones,alicejones@example.com,555-3456
4,5,David,Brown,davidbrown@example.com,555-7890
5,6,Emily,Davis,emilydavis@example.com,555-2345
6,7,Frank,Wilson,frankwilson@example.com,555-6789
7,8,Grace,Lee,gracelee@example.com,555-1234
8,9,Henry,Chen,henrychen@example.com,555-5678
9,10,Isabel,Garcia,isabelgarcia@example.com,555-9012


In [25]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT CONCAT(c.first_name, ' ', c.last_name) as customer_name, SUM(s.total_price) as total_sales
FROM sales_data.sales s
JOIN sales_data.customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df_customer = pd.read_sql_query(query, connection)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28440\3188071144.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [26]:
df_customer

,customer_name,total_sales
0,John Doe,32970.0
1,David Brown,32140.0
2,Bob Smith,30415.0
3,Frank Wilson,30410.0
4,Alice Jones,27980.0
5,Jane Doe,27590.0
6,Emily Davis,26120.0
7,Grace Lee,25480.0
8,Isabel Garcia,24785.0
9,Henry Chen,23175.0


In [31]:
# Create the Plotly figure
fig = px.bar(df_customer,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
    
)
fig.update_xaxes(tickangle=10)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

>## Create PDF Report

In [33]:
# Define the font color as RGB values (dark gray)
from fpdf import FPDF  
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

''